In [8]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import pandas as pd
import time
import tqdm

import warnings
warnings.filterwarnings("ignore")

squad_codes = pd.read_csv('big5_MLS_squad_codes.csv')


In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
#lists to loop through
cats = ['schedule','shooting','keeper','passing','passing_types','gca','defense','possession','misc']
year_range = ['2017-2018','2018-2019','2019-2020','2020-2021','2021-2022','2022-2023']

In [20]:
opp_score_columns

[['oppDate',
  'oppTime',
  'oppComp',
  'oppRound',
  'oppDay',
  'oppVenue',
  'oppResult',
  'oppGF',
  'oppGA',
  'oppOpponent',
  'oppxG',
  'oppxGA',
  'oppPossession%',
  'oppAttendance',
  'oppCaptain',
  'oppFormation',
  'oppReferee',
  'oppMatch Report',
  'oppNotes']]

In [4]:
#FB Ref Columns


score_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','xG','xGA','Possession%','Attendance','Captain','Formation','Referee','Match Report','Notes']]

shooting_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','Gls','Sh','SoT','SoT%','G/Sh','G/SoT','Avg Shot Distance','Free Kick Shots','Penalties Made','PKatt','xG','Non pen xG','npxG/Sh','G minus xG','npxG-xG','Match Report']]

goalkeeping_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','SoTA','GA','Saves','Save%','Clean Sheet','Post-ShotxG','PSxGPlusMinus','PKatt','PKAllowed','PKsaved','PKmissed','Passes40ydspluscompleted','Passes40ydsplusAttempted','Passes40ydsplusCmp%','PassesAtt','PassesThrwn','%ofPassesLaunched','AvgPassLen','GoalKicksAtt','GoalKicks%Launched','GoalKicksAvgLen','OppCrossesAtt','OppCrossesStp','OppCrossesStp%','# of Sweeper Actions','AvgDist sweeper Actions','Match Report']]

passing_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','Passes Completed','Passes Attempted','Pass Completion %','Total Pass Dist','Total Pass Dist Twd Goal','Short Pass Comp','Short Pass Att','Shrt Pass Cmp%','Med Pass Comp','Med Pass Att','Med Pass Cmp %','Lng Pass Cmp','Lng Pass Att','Lng Pass Cmp %','Ast','xAssistedGoals','xAssists','KeyPasses','Attacking Third Passes','Passes Into 18 yd box Comp','Completed Cross','Prog Passes past own 40 yd line','Match Report']]

pass_type_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','Passes Attempted','Live Passes','Deadball passes','Free Kick Passes','Passes between open defenders','Passes that go 40 plus yds wide','Crosses','Throw Ins','Corner Kicks','Inswinger Corners','Outswinging Corners','Straight Corners','Passes Completed','Offsides','Passes Blocked','Match Report']]

goal_shot_creation_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','Shot Creating Actions','Live Passes Leading to shot','Deadball Passes Leading to shot','Dribbles leading to shot','Shots leading to another shot','Fouls drawn leading to shot','Def Actions Leading to Shot','Goal Creating Actions','Passes leading to goal','Goals from deadball','Goals from dribble','Goals from other shot','Goals from foul drawn','Goals from defensive action','Match Report']]

defensive_action_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','Tackles','TacklesWon','Tackles in Def 3rd','Tackles in Mid 3rd','Tackles in Att 3rd','Tackles vs Dribble','Tackles vs Dribble Att','% of Dribblers tackled succesful','# of times dribbled past','Blocks','Shots Blocked','Passes Blocked','Interceptions','Tackles + Int','Clearances','Errors','Match Report']]

possession_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','Possession','Touches','Touches in Def Pen Area','Touches in Def 3rd','Touches in Mid 3rd','Touches in Att 3rd','Touches in Att Pen Area','Live ball touches','Attempted Take ons','Succesful Take Ons','Take on success rate','Number of times tackled during take on','% of take ons tackled','Carries','Total Carry Distance','Progressive Carrying Distance','Progressive Carries','Carries into final third','Carries into penalty area','Miscontrols','Dispossessions','Passes Received','Progressive Passes Received','Match Report']]

misc_stats_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','CrdY','CrdR','2CrdY','Fls','Fld','Off','Crs','Int','TklW','PKwon','PKcon','OG','Recov','Won','Lost','Won%','Match Report']]

opp_score_columns = [['opp' + x for x in score_columns[0]]]

opp_shooting_columns = [['opp' + x for x in shooting_columns[0]]]

opp_goalkeeping_columns = [['opp' + x for x in goalkeeping_columns[0]]]

opp_passing_columns = [['opp' + x for x in passing_columns[0]]]

opp_pass_type_columns = [['opp' + x for x in pass_type_columns[0]]]

opp_goal_shot_creation_columns = [['opp' + x for x in goal_shot_creation_columns[0]]]

opp_defensive_action_columns = [['opp' + x for x in defensive_action_columns[0]]]

opp_possession_columns = [['opp' + x for x in possession_columns[0]]]

opp_misc_stats_columns = [['opp' + x for x in misc_stats_columns[0]]]

In [5]:
# Score Links
score_links = []

for year in range(len(year_range)):
    for squad in range(len(squad_codes)):
        link = (f"https://fbref.com/en/squads/{squad_codes['Squad_Code'][squad]}/{year_range[year]}/matchlogs/all_comps/{cats[0]}/{squad_codes['Text'][squad]}")
        score_links.append(link)
        
# Shooting Links
shooting_links = []

for year in range(len(year_range)):
    for squad in range(len(squad_codes)):
        link = (f"https://fbref.com/en/squads/{squad_codes['Squad_Code'][squad]}/{year_range[year]}/matchlogs/all_comps/{cats[1]}/{squad_codes['Text'][squad]}")
        shooting_links.append(link)
        
# Goalkeeping Links
goalkeeping_links = []

for year in range(len(year_range)):
    for squad in range(len(squad_codes)):
        link = (f"https://fbref.com/en/squads/{squad_codes['Squad_Code'][squad]}/{year_range[year]}/matchlogs/all_comps/{cats[2]}/{squad_codes['Text'][squad]}")
        goalkeeping_links.append(link)
        
# Passing Links
passing_links = []

for year in range(len(year_range)):
    for squad in range(len(squad_codes)):
        link = (f"https://fbref.com/en/squads/{squad_codes['Squad_Code'][squad]}/{year_range[year]}/matchlogs/all_comps/{cats[3]}/{squad_codes['Text'][squad]}")
        passing_links.append(link)
        
# Pass Type Links
pass_type_links = []

for year in range(len(year_range)):
    for squad in range(len(squad_codes)):
        link = (f"https://fbref.com/en/squads/{squad_codes['Squad_Code'][squad]}/{year_range[year]}/matchlogs/all_comps/{cats[4]}/{squad_codes['Text'][squad]}")
        pass_type_links.append(link)
        
# Goal Shot Creation Links
gca_links = []

for year in range(len(year_range)):
    for squad in range(len(squad_codes)):
        link = (f"https://fbref.com/en/squads/{squad_codes['Squad_Code'][squad]}/{year_range[year]}/matchlogs/all_comps/{cats[5]}/{squad_codes['Text'][squad]}")
        gca_links.append(link)
        
# Defensive Action Links
defense_links = []

for year in range(len(year_range)):
    for squad in range(len(squad_codes)):
        link = (f"https://fbref.com/en/squads/{squad_codes['Squad_Code'][squad]}/{year_range[year]}/matchlogs/all_comps/{cats[6]}/{squad_codes['Text'][squad]}")
        defense_links.append(link)
        
# Possession Links
poss_links = []

for year in range(len(year_range)):
    for squad in range(len(squad_codes)):
        link = (f"https://fbref.com/en/squads/{squad_codes['Squad_Code'][squad]}/{year_range[year]}/matchlogs/all_comps/{cats[7]}/{squad_codes['Text'][squad]}")
        poss_links.append(link)
        
# Misc Links
misc_links = []

for year in range(len(year_range)):
    for squad in range(len(squad_codes)):
        link = (f"https://fbref.com/en/squads/{squad_codes['Squad_Code'][squad]}/{year_range[year]}/matchlogs/all_comps/{cats[8]}/{squad_codes['Text'][squad]}")
        misc_links.append(link)
        
# Test links 
test_links = poss_links[0:4]

In [6]:
# Team & Opp Dataframes
score_df = pd.DataFrame(columns=score_columns, dtype=object)
shooting_df = pd.DataFrame(columns=shooting_columns, dtype=object)
goalkeeping_df = pd.DataFrame(columns=goalkeeping_columns, dtype=object)
passing_df = pd.DataFrame(columns=passing_columns, dtype=object)
pass_type_df = pd.DataFrame(columns=pass_type_columns, dtype=object)
goal_shot_creation_df = pd.DataFrame(columns=goal_shot_creation_columns, dtype=object)
defensive_action_df = pd.DataFrame(columns=defensive_action_columns, dtype=object)
possession_df = pd.DataFrame(columns=possession_columns, dtype=object)
misc_stats_df = pd.DataFrame(columns=misc_stats_columns, dtype=object)

opp_score_df = pd.DataFrame(columns=opp_score_columns, dtype=object)
opp_shooting_df = pd.DataFrame(columns=opp_shooting_columns, dtype=object)
opp_goalkeeping_df = pd.DataFrame(columns=opp_goalkeeping_columns, dtype=object)
opp_passing_df = pd.DataFrame(columns=opp_passing_columns, dtype=object)
opp_pass_type_df = pd.DataFrame(columns=opp_pass_type_columns, dtype=object)
opp_goal_shot_creation_df = pd.DataFrame(columns=opp_goal_shot_creation_columns, dtype=object)
opp_defensive_action_df = pd.DataFrame(columns=opp_defensive_action_columns, dtype=object)
opp_possession_df = pd.DataFrame(columns=opp_possession_columns, dtype=object)
opp_misc_stats_df = pd.DataFrame(columns=opp_misc_stats_columns, dtype=object)

test_df = pd.DataFrame(columns=possession_columns, dtype=object)
test_2_df = pd.DataFrame(columns=opp_possession_columns, dtype=object)

In [7]:
# This works! Practice Example with limited amount of links - just need to change links list, test_df, and columns

# Loop through each link
for link in tqdm.tqdm(test_links):
    time.sleep(3)
    
    try:
        # Use requests to get the HTML content of the page
        res = requests.get(link)
        soup = BeautifulSoup(res.text, 'html.parser')

        # Find the header for team info
        header_data = soup.find('th', {'data-stat': 'header_for_against'}).text

        # Find the table with the specified div id
        table = soup.find('table', {'id': 'matchlogs_for'})

        # Find table body
        table_body = table.find('tbody')

        # Find all the rows in the table
        rows = table_body.find_all('tr')

        # Initialize an empty list to store the data
        data = []

        # Loop through each row and extract the data
        for row in rows:
            cols = row.find_all(['td','th'])
            cols = [col.text.strip() for col in cols]
            data.append(cols)

        # Create a temporary dataframe from the extracted data
        temp_df = pd.DataFrame(data, columns=possession_columns)
        temp_df['Team'] = header_data
        test_df = test_df.append(temp_df, ignore_index=True)
    
    except:
        continue
        
    
        

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]C:\Users\tbrec\AppData\Local\Temp\ipykernel_25856\3022965613.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_df = test_df.append(temp_df, ignore_index=True)
 25%|█████████████████████                                                               | 1/4 [00:03<00:09,  3.25s/it]C:\Users\tbrec\AppData\Local\Temp\ipykernel_25856\3022965613.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_df = test_df.append(temp_df, ignore_index=True)
 50%|██████████████████████████████████████████                                          | 2/4 [00:06<00:06,  3.26s/it]C:\Users\tbrec\AppData\Local\Temp\ipykernel_25856\3022965613.py:36: FutureWarning: The frame.append method is deprecated and will b

In [11]:
test_df

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Progressive Carrying Distance,Progressive Carries,Carries into final third,Carries into penalty area,Miscontrols,Dispossessions,Passes Received,Progressive Passes Received,Match Report,Team
0,2017-07-28,20:00,Ligue 2,Matchweek 1,Fri,Away,D,0,0,Niort,...,,,,,,,,,Match Report,For Ajaccio
1,2017-08-04,20:00,Ligue 2,Matchweek 2,Fri,Home,W,2,1,Brest,...,,,,,,,,,Match Report,For Ajaccio
2,2017-08-08,21:00,Coupe de la Ligue,First round,Tue,Away,L,1,2,Lens,...,,,,,,,,,Match Report,For Ajaccio
3,2017-08-11,20:00,Ligue 2,Matchweek 3,Fri,Away,D,0,0,Orléans,...,,,,,,,,,Match Report,For Ajaccio
4,2017-08-18,20:00,Ligue 2,Matchweek 4,Fri,Home,W,2,0,Paris FC,...,,,,,,,,,Match Report,For Ajaccio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,2018-04-29,16:30,Premier League,Matchweek 36,Sun,Away,L,1,2,Manchester Utd,...,1187,24,23,3,11,12,377,36,Match Report,For Arsenal
185,2018-05-03,21:05,Europa Lg,Semi-finals,Thu,Away,L,0,1,es Atlético Madrid,...,1099,29,15,3,14,7,442,47,Match Report,For Arsenal
186,2018-05-06,16:30,Premier League,Matchweek 37,Sun,Home,W,5,0,Burnley,...,1875,36,24,13,11,10,606,64,Match Report,For Arsenal
187,2018-05-09,19:45,Premier League,Matchweek 31,Wed,Away,L,1,3,Leicester City,...,998,28,14,8,8,11,408,35,Match Report,For Arsenal


In [9]:
#Test to get opponent info
# Loop through each link
for link in tqdm.tqdm(test_links):
    time.sleep(3)
    
    try:
        # Use requests to get the HTML content of the page
        res = requests.get(link)
        soup = BeautifulSoup(res.text, 'html.parser')

        # Find the header for team info
        header_data = soup.find('th', {'data-stat': 'header_for_against'}).text

        # Find the table with the specified div id
        table = soup.find('table', {'id': 'matchlogs_against'})

        # Find table body
        table_body = table.find('tbody')

        # Find all the rows in the table
        rows = table_body.find_all('tr')

        # Initialize an empty list to store the data
        data = []

        # Loop through each row and extract the data
        for row in rows:
            cols = row.find_all(['td','th'])
            cols = [col.text.strip() for col in cols]
            data.append(cols)

        # Create a temporary dataframe from the extracted data
        temp_df = pd.DataFrame(data, columns=opp_possession_columns)
        temp_df['Team'] = header_data
        test_2_df = test_2_df.append(temp_df, ignore_index=True)
    
    except:
        continue
        

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:13<00:00,  3.25s/it]


In [10]:
test_2_df

,oppDate,oppTime,oppComp,oppRound,oppDay,oppVenue,oppResult,oppGF,oppGA,oppOpponent,...,oppProgressive Carrying Distance,oppProgressive Carries,oppCarries into final third,oppCarries into penalty area,oppMiscontrols,oppDispossessions,oppPasses Received,oppProgressive Passes Received,oppMatch Report,Team
0,2017-07-28,20:00,Ligue 2,Matchweek 1,Fri,Home,D,0,0,Niort,...,,,,,,,,,Match Report,For Ajaccio
1,2017-08-04,20:00,Ligue 2,Matchweek 2,Fri,Away,L,1,2,Brest,...,,,,,,,,,Match Report,For Ajaccio
2,2017-08-08,21:00,Coupe de la Ligue,First round,Tue,Home,W,2,1,Lens,...,,,,,,,,,Match Report,For Ajaccio
3,2017-08-11,20:00,Ligue 2,Matchweek 3,Fri,Home,D,0,0,Orléans,...,,,,,,,,,Match Report,For Ajaccio
4,2017-08-18,20:00,Ligue 2,Matchweek 4,Fri,Away,L,0,2,Paris FC,...,,,,,,,,,Match Report,For Ajaccio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,2018-04-29,16:30,Premier League,Matchweek 36,Sun,Home,W,2,1,Manchester Utd,...,1565,31,25,8,17,17,575,57,Match Report,For Arsenal
185,2018-05-03,21:05,Europa Lg,Semi-finals,Thu,Home,W,1,0,es Atlético Madrid,...,824,17,17,5,11,22,359,37,Match Report,For Arsenal
186,2018-05-06,16:30,Premier League,Matchweek 37,Sun,Away,L,0,5,Burnley,...,634,10,8,3,11,9,310,30,Match Report,For Arsenal
187,2018-05-09,19:45,Premier League,Matchweek 31,Wed,Home,W,3,1,Leicester City,...,908,24,15,8,8,13,264,41,Match Report,For Arsenal
